In [42]:
import imagenetscraper
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import vgg16 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.utils import np_utils, to_categorical
from keras import backend as K

%matplotlib inline

In [2]:
vgg_model = vgg16.VGG16(weights='imagenet',
                        include_top=False,
                        input_shape=(224, 224, 3))

In [3]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Freeze the Required Layers

In [4]:
vgg_model.layers[:-4]

In [5]:
# freeze the layers except the last 4 layers
for layer in vgg_model.layers[:-4]:
    layer.trainable = False
    
# check the trainable status of the individual layers
for layer in vgg_model.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x1823ed8f98> False
<keras.layers.convolutional.Conv2D object at 0x1823eed4e0> False
<keras.layers.convolutional.Conv2D object at 0x1823eed438> False
<keras.layers.pooling.MaxPooling2D object at 0x1823f0bd68> False
<keras.layers.convolutional.Conv2D object at 0x1823eedb00> False
<keras.layers.convolutional.Conv2D object at 0x1823f3afd0> False
<keras.layers.pooling.MaxPooling2D object at 0x1823f28278> False
<keras.layers.convolutional.Conv2D object at 0x1823f50860> False
<keras.layers.convolutional.Conv2D object at 0x1131d16d8> False
<keras.layers.convolutional.Conv2D object at 0x1823f61f98> False
<keras.layers.pooling.MaxPooling2D object at 0x1823f977f0> False
<keras.layers.convolutional.Conv2D object at 0x1823fa8438> False
<keras.layers.convolutional.Conv2D object at 0x1823fd2438> False
<keras.layers.convolutional.Conv2D object at 0x1823fc0f28> False
<keras.layers.pooling.MaxPooling2D object at 0x1823ff4ef0> False
<keras.layers.convolutional

## Create A New Model

In [6]:
from keras import models
from keras import layers
from keras import optimizers

In [7]:
# create the model
model_ftuned = models.Sequential()

In [8]:
# add the VGG convolutional base model
model_ftuned.add(vgg_model)

# add new layers
model_ftuned.add(layers.Flatten())
model_ftuned.add(layers.Dense(1024, activation='relu'))
model_ftuned.add(layers.Dropout(0.5))
model_ftuned.add(layers.Dense(3, activation='softmax'))

# show a summary of the model and check the number of trainable parameters
model_ftuned.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 40,408,899
Trainable params: 32,773,635
Non-trainable params: 7,635,264
_________________________________________________________________


## Setup The Data Generators

In [9]:
train_dir = './datasets/train/'
validation_dir = './datasets/validation/'
 
n_train = 600 # of total 3302
n_validation = 160 # of total 824

In [10]:
# change the batchsize according to your system RAM
train_batch_size = 100
validation_batch_size = 10

#### Train dataset

In [19]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_images = np.zeros(shape=(n_train, 224, 224, 3)) #(600, 224, 224, 3)
train_labels = np.zeros(shape=(n_train, 3))

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batch_size,
        class_mode='categorical')

Found 3302 images belonging to 3 classes.


In [22]:
i = 0
for inputs_batch, labels_batch in train_generator:    
    train_images[i * train_batch_size: (i+1) * train_batch_size] = inputs_batch
    train_labels[i * train_batch_size : (i+1) * train_batch_size] = labels_batch
    
    i+=1
    if i * train_batch_size >= n_train:
        break

In [23]:
train_images = np.reshape(train_images, (n_train, 224 * 224 * 3))

#### Validation dataset

In [26]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_images = np.zeros(shape=(n_validation, 224, 224, 3)) #(600, 224, 224, 3)
validation_labels = np.zeros(shape=(n_validation, 3))

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=validation_batch_size,
        class_mode='categorical',
        shuffle=False)

Found 824 images belonging to 3 classes.


In [27]:
i = 0
for inputs_batch, labels_batch in validation_generator:    
    validation_images[i * validation_batch_size: (i+1) * validation_batch_size] = inputs_batch
    validation_labels[i * validation_batch_size : (i+1) * validation_batch_size] = labels_batch

    i+=1
    if i * validation_batch_size >= n_validation:
        break

In [28]:
validation_images = np.reshape(validation_images, (n_validation, 224 * 224 * 3))

#### Change the labels from integer to categorical data

In [55]:
train_labels_one_hot = to_categorical(train_labels)
validation_labels_one_hot = to_categorical(validation_labels)
 
# display the change for category label using one-hot encoding
print('Original label 0 : ', train_labels[6])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot[6])

Original label 0 :  [0. 1. 0.]
After conversion to categorical ( one-hot ) :  [[1. 0.]
 [0. 1.]
 [1. 0.]]


## Train The Model

In [57]:
# compile the model
model_ftuned.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.RMSprop(lr=1e-4),
                      metrics=['acc'])

*There is a bug of shaping size for feeding to the network. Fix it first!*

In [ ]:
data_augmentated = ImageDataGenerator(
                    rotation_range=10,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.,
                    zoom_range=1.,
                    horizontal_flip=True,
                    vertical_flip=False)

# train the model
history_ftuned = model_ftuned.fit_generator(data_augmentated.flow(train_images, train_labels_one_hot, batch_size=20),
                                     steps_per_epoch=int(np.ceil(train_images.shape[0] / float(20))),
                                     epochs=epochs,
                                     validation_data=(validation_images, validation_labels_one_hot),
                                     workers=4)

## Check Performance

In [ ]:
epochs = range(len(history_ftuned.history['acc']))

In [ ]:
# plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(epochs, history_ftuned.history['loss'],'r',linewidth=3.0)
plt.plot(epochs, history_ftuned.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [ ]:
# plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(epochs, history_ftuned.history['acc'],'r',linewidth=3.0)
plt.plot(epochs, history_ftuned.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

## Save The Model

In [ ]:
model_ftuned.save('models/vgg_fcl_fine_tuned.h5')

<hr/>